# Generate shape coefficients

The following steps are performed to obtain all the artifacts for shape coefficient use case:

1. Read parameters file
2. Read Mesh description
   * Its path can be specified
   * Or it can be obtained from the simulation configuration file
3. Read data for pressure coefficient

Initialize Path Manager

In [1]:
from cfdmod.use_cases.pressure.path_manager import CePathManager
import pathlib

path_manager = CePathManager(output_path=pathlib.Path("./output/pressure"))

Read post-processing Ce config file

In [2]:
from cfdmod.use_cases.pressure.shape.Ce_config import CeConfig

config_path=pathlib.Path("./fixtures/tests/pressure/Ce_params.yaml")
post_proc_cfg = CeConfig.from_file(config_path)

post_proc_cfg

{'pattern_1': CeConfig(zoning=ZoningConfig(global_zoning=ZoningModel(x_intervals=[0.0, 25.0, 50.0, 75.0, 100.0, 125.0, 150.0, 175.0, 200.0, 225.0, 250.0], y_intervals=[0.0, 25.0, 50.0, 75.0, 100.0, 125.0, 150.0, 175.0, 200.0], z_intervals=[0.0, 5.0, 10.0, 15.0, 20.0]), no_zoning=['sfc1'], exclude=['sfc1'], exceptions={'zoning1': ExceptionZoningModel(x_intervals=[0.0, 25.0, 50.0, 75.0, 100.0, 125.0, 150.0, 175.0, 200.0, 225.0, 250.0], y_intervals=[0.0, 25.0, 50.0, 75.0, 100.0, 125.0, 150.0, 175.0, 200.0], z_intervals=[0.0, 20.0], surfaces=['sfc1'])}), statistics=['min', 'max', 'std', 'avg', 'skewness', 'kurtosis']),
 'pattern_2': CeConfig(zoning=ZoningConfig(global_zoning=ZoningModel(x_intervals=[0.0, 250.0], y_intervals=[0.0, 200.0], z_intervals=[0.0, 20.0]), no_zoning=[], exclude=[], exceptions={}), statistics=['min', 'max', 'std', 'avg', 'skewness', 'kurtosis']),
 'pattern_3': CeConfig(zoning=ZoningConfig(global_zoning=ZoningModel(x_intervals=[0.0, 125.0, 250.0], y_intervals=[-inf, i

Normalize LNAS Mesh [Example]

In [3]:
from nassu.lnas import LagrangianFormat

original_mesh_path = pathlib.Path("./fixtures/tests/pressure/galpao/galpao.lnas")
original_mesh = LagrangianFormat.from_file(original_mesh_path)

new_verts = original_mesh.geometry.vertices.copy()

new_verts[:, 0] -= new_verts[:, 0].min()
new_verts[:, 1] -= new_verts[:, 1].min()
new_verts[:, 2] -= new_verts[:, 2].min()

original_mesh.geometry.vertices = new_verts

original_mesh.to_file(original_mesh_path.parent / f"{original_mesh.name}.normalized.lnas")

print(original_mesh.name, original_mesh.geometry.vertices.shape, original_mesh.geometry.triangles.shape)

original_mesh.geometry.vertices[:, 0].min(), original_mesh.geometry.vertices[:, 1].min(), original_mesh.geometry.vertices[:, 2].min()

galpao (1549, 3) (2915, 3)


(0.0, 0.0, 0.0)

Read Normalized LNAS Mesh

In [4]:
from nassu.lnas import LagrangianFormat

mesh_path=pathlib.Path("./fixtures/tests/pressure/galpao/galpao.normalized.lnas")
mesh = LagrangianFormat.from_file(mesh_path)

mesh.name, mesh.geometry.vertices.shape, mesh.geometry.triangles.shape

('galpao', (1549, 3), (2915, 3))

Read Pressure Coefficient data

In [5]:
import pandas as pd

cp_data_path=pathlib.Path("./fixtures/tests/pressure/data/cp_t.resampled.hdf")
cp_data = pd.read_hdf(cp_data_path)

cp_data_to_use = (
    cp_data.to_frame() if isinstance(cp_data, pd.Series) else cp_data
)

Add region index to pressure coefficient data

In [6]:
from cfdmod.api.vtk.write_vtk import write_polydata, merge_polydata
from cfdmod.logger import logger
from cfdmod.use_cases.pressure.shape.Ce_data import process_surface

n_timesteps = cp_data_to_use["time_step"].unique().shape[0]

for cfg_label, cfg in post_proc_cfg.items():
    processed_polydata = []
    logger.info(f"Processing {cfg_label} ...")
    for sfc in mesh.surfaces.keys():
        if sfc in cfg.zoning.exclude:
            logger.info(f"Surface {sfc} ignored!")  # Ignore surface
            continue

        logger.info(f"Processing surface {sfc} ...")

        processed_surface = process_surface(
            body_mesh=mesh,
            sfc_label=sfc,
            cfg=cfg,
            cp_data=cp_data_to_use,
            n_timesteps=n_timesteps,
        )
        
        processed_polydata.append(processed_surface.polydata)

        logger.info(f"Processed surface {sfc}")

    # Output 5: VTK
    merged_polydata = merge_polydata(processed_polydata)
    write_polydata(path_manager.get_vtp_path(mesh.name, cfg_label), merged_polydata)

    logger.info(f"Merged polydata for {cfg_label}")

[2023-10-20 15:48:58,176] [INFO] - cfdmod - Processing pattern_1 ... (2943861426.py:9)
[2023-10-20 15:48:58,177] [INFO] - cfdmod - Processing surface L1_xp ... (2943861426.py:15)
[2023-10-20 15:48:58,215] [INFO] - cfdmod - Processed surface L1_xp (2943861426.py:27)
[2023-10-20 15:48:58,216] [INFO] - cfdmod - Processing surface L2_yp ... (2943861426.py:15)
[2023-10-20 15:48:58,385] [INFO] - cfdmod - Processed surface L2_yp (2943861426.py:27)
[2023-10-20 15:48:58,386] [INFO] - cfdmod - Processing surface L3_zp_yp ... (2943861426.py:15)
[2023-10-20 15:48:58,455] [INFO] - cfdmod - Processed surface L3_zp_yp (2943861426.py:27)
[2023-10-20 15:48:58,455] [INFO] - cfdmod - Processing surface L4_zp_ym ... (2943861426.py:15)
[2023-10-20 15:48:58,516] [INFO] - cfdmod - Processed surface L4_zp_ym (2943861426.py:27)
[2023-10-20 15:48:58,517] [INFO] - cfdmod - Processing surface L5_ym ... (2943861426.py:15)
[2023-10-20 15:48:58,660] [INFO] - cfdmod - Processed surface L5_ym (2943861426.py:27)
[2023-